# G2Config Reference

More information:

1. [GitHub repository](https://github.com/Senzing/docker-jupyter)
1. [Senzing documentation](http://docs.senzing.com/?python#g2config)

## Prepare Environment

In [ ]:
import com.senzing.g2.engine.G2Config;
import com.senzing.g2.engine.G2ConfigJNI;

### Helper class for Json Rendering

In [ ]:
%%loadFromPOM
<dependency>
    <groupId>org.glassfish</groupId>
    <artifactId>javax.json</artifactId>
    <version>1.1.4</version>
</dependency>

In [ ]:
import javax.json.*;
import static java.util.Collections.*;
import static javax.json.stream.JsonGenerator.PRETTY_PRINTING;

In [ ]:
public class JsonUtil {
    private static final JsonWriterFactory PRETTY_FACTORY
        = Json.createWriterFactory(singletonMap(PRETTY_PRINTING, true));
        
    private static final JsonWriterFactory UGLY_FACTORY
        = Json.createWriterFactory(emptyMap());
    
    public static String toJsonText(JsonValue val) {
        return toJsonText(val, true);
    }

    public static String toJsonText(JsonValue val, boolean prettyPrint) {
        JsonWriterFactory factory = (prettyPrint) ? PRETTY_FACTORY : UGLY_FACTORY;
        StringWriter sw = new StringWriter();
        JsonWriter writer = factory.createWriter(sw);
        writer.write(val);
        sw.flush();
        return sw.toString();
    }
    
    public static JsonObject parseJsonObject(String jsonText) {
        if (jsonText == null) return null;
        StringReader sr = new StringReader(jsonText);
        JsonReader jsonReader = Json.createReader(sr);
        return jsonReader.readObject();
  }

    public static JsonArray parseJsonArray(String jsonText) {
        if (jsonText == null) return null;
        StringReader sr = new StringReader(jsonText);
        JsonReader jsonReader = Json.createReader(sr);
        return jsonReader.readArray();
  }
    
    
}

In [ ]:
import java.util.UUID;
public static void RenderJSON(String str){
    JsonObject json = JsonUtil.parseJsonObject(str);
    String Config = JsonUtil.toJsonText(json, false);
    UUID id = UUID.randomUUID();
    String uuid = id.toString();
    String div = "<div id=\""+ uuid +"\" style=\"height:100%; width:100%; background-color: LightCyan\"></div>";
    display(div, "text/html");
    String jav = "require([\"https://rawgit.com/caldwell/renderjson/master/renderjson.js\"], function() {document.getElementById(\'"+ uuid +"\').appendChild(renderjson("+json+"))});";
    display(jav, "application/javascript");
}

### Initialize Senzing configuration

Using environment variables and default values, create `senzingConfigJson`.
This value is used when instantiating Senzing objects.

In [ ]:
// Get variables used in constructing Senzing Engine configuration.

String configPath = System.getenv("SENZING_ETC_DIR");
if (configPath == null) {
    configPath = "/etc/opt/senzing";
}

String supportPath = System.getenv("SENZING_DATA_VERSION_DIR");
if (supportPath == null) {
    supportPath = "/opt/senzing/data";
}

String g2Path = System.getenv("SENZING_G2_DIR");
if (g2Path == null) {
    g2Path = "/opt/senzing/g2";
}

String resourcePath = g2Path + "/resources";

String sqlConnection = System.getenv("SENZING_SQL_CONNECTION");
if (sqlConnection == null) {
    sqlConnection = "sqlite3://na:na@/var/opt/senzing/sqlite/G2C.db";
}

// Construct the JSON string used for Senzing Engine configuration.

String senzingConfigJson = "{"
   + "\"PIPELINE\": {"
   +     "\"CONFIGPATH\": \"" + configPath + "\","
   +     "\"SUPPORTPATH\": \"" + supportPath + "\","
   +     "\"RESOURCEPATH\": \"" + resourcePath + "\""
   + "},"
   + "\"SQL\": {"
   +     "\"CONNECTION\": \"" + sqlConnection + "\""
   + "}}";
   
System.out.println(senzingConfigJson);

## G2Config

The G2Config API is used to create a new JSON configuration from the default template
or manipulate a JSON configuration that it is provided.
The API read/writes a JSON string, but that string would typically be
read/written from/to a g2config.json file that the G2 engine reads during the initV2(...) call.

### G2Config Initialization

To start using G2Config, you must first create and initialize the G2Config object.
This should be done once per process.

Create a new instance of the Senzing G2Config object and assign it to a variable.
Then, call the appropriate initialization method (such as initV2) to initialize the G2Config object.

During the call, the initialization methods accept the following parameters:

- **module_name:** A short name given to this instance of the G2 engine (i.e. your G2Module object)
- **senzing_config_json:** A JSON document containing system parameters (see the section called "Initialization Parameters")
- **verbose_logging:** A boolean which enables diagnostic logging - this will print a massive amount of information to stdout (default = False)

Calling these functions will return "0" upon success - useful for error handling.

In [ ]:
String moduleName = "ExampleG2Config";
boolean verboseLogging = true;

G2Config g2config = new G2ConfigJNI();
int return_code = g2config.initV2(moduleName, senzingConfigJson, verboseLogging);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(return_code);

### Create Configuration handle

In [ ]:
long config_handle = g2config.create()

## Datasources

### listDataSources
Call G2Config's `listDataSourcesV2()` method and print results.

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2config.listDataSourcesV2(config_handle, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

### addDataSource
Call G2Config's `addDataSourceV2()` method and print results

In [ ]:
String dataSource = "{\"DSRC_CODE\": \"CUSTOMER\"}";

StringBuffer response = new StringBuffer();

int return_code = g2config.addDataSourceV2(config_handle, dataSource, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

### listDataSources again
Call G2Config's `listDataSourcesV2()` method and print results. Notice that the list now contains the newly add dataSource of "CUSTOMER".

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2config.listDataSourcesV2(config_handle, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

## Entities

### listEntityClasses
Call G2Config's `listEntityClassesV2()` method and print results.

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2config.listEntityTypesV2(config_handle, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

### listEntityTypes
Call G2Config's `listEntityTypesV2()` method and print results.

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2config.listEntityTypesV2(config_handle, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

### addEntityClass
Call G2Config's `addEntityClassV2()` method and print results

In [ ]:
String entityClass = "{\"ECLASS_CODE\": \"COMPANY\"}";
StringBuffer response = new StringBuffer();

int return_code = g2config.addEntityClassV2(config_handle, entityClass, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

### listEntityClasses again
Call G2Config's `listEntityClassesV2()` method and print results. Notice that the list now contains the newly added entityClass "COMPANY".

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2config.listEntityClassesV2(config_handle, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

### addEntityType
Call G2Config's `addEntityTypeV2()` method and print results.

In [ ]:
String entityType = "{\"ETYPE_CODE\": \"PROFESSIONAL\", \"ECLASS_CODE\": \"ACTOR\"}";
StringBuffer response = new StringBuffer();

int return_code = g2config.addEntityTypeV2(config_handle, entityType, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

### listEntityTypes again
Call G2Config's `listEntityTypesV2()` method and print results. Notice that the list now contains the newly added entityType "PROFESSIONAL".

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2config.listEntityTypesV2(config_handle, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

### deleteEntityClass
Call G2Config's `deleteEntityClassV2()` method. Returns 0 on success

In [ ]:
String entityClass = "{\"ECLASS_CODE\": \"COMPANY\"}";

int return_code = g2config.deleteEntityClassV2(config_handle, entityClass);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(return_code);

### deleteEntityType
Call G2Config's `deleteEntityTypeV2()` method. Returns 0 on success

In [ ]:
String entityType = "{\"ETYPE_CODE\": \"PROFESSIONAL\", \"ECLASS_CODE\": \"ACTOR\"}";

int return_code = g2config.deleteEntityTypeV2(config_handle, entityType);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(return_code);

## Feature

### listFeatureElements
Call G2Config's `listFeatureElementsV2()` method and print results.

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2config.listFeatureElementsV2(config_handle, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

### getFeatureElement
Call G2Config's `getFeatureElementV2()` method and print results.

In [ ]:
String featureElm = "{\"FELEM_CODE\": \"GIVEN_NAME\"}";
StringBuffer response = new StringBuffer();

int return_code = g2config.getFeatureElementV2(config_handle, featureElm, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

### addFeatureElement
Call G2Config's `addFeatureElementV2()` method and print results.

In [ ]:
String featureElm = "{\"FELEM_CODE\":\"myNewFelem\",\"FELEM_ID\":789,\"TOKENIZE\":1,\"DATA_TYPE\":\"string\"}";
StringBuffer response = new StringBuffer();

int return_code = g2config.addFeatureElementV2(config_handle, featureElm, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

### deleteFeatureElement
Call G2Config's `deleteFeatureElementV2()` method. Returns 0 on success

In [ ]:
String featureElm = "{\"FELEM_CODE\": \"myNewFelem\"}";

int return_code = g2config.deleteFeatureElementV2(config_handle, featureElm);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(return_code);

### listFeatureClasses
Call G2Config's `listFeatureClassesV2()` method and print results.

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2config.listFeatureClassesV2(config_handle, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

### listFeatures
Call G2Config's `listFeaturesV2()` method and print results.

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2config.listFeaturesV2(config_handle, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

### getFeature
Call G2Config's `getFeatureV2()` method and print results.

In [ ]:
String feature = "{\"FTYPE_CODE\":\"name\"}";
StringBuffer response = new StringBuffer();

int return_code = g2config.getFeatureV2(config_handle, feature, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

### addFeature
Call G2Config's `addFeatureV2()` method and print results.

In [ ]:
String feature = "{\"FTYPE_CODE\":\"myNewFtype\",\"FTYPE_ID\":789,\"FCLASS_CODE\":\"ELECTED_ID\",\"FTYPE_FREQ\":\"FF\", \"FEATURE_ELEMENTS\":[{\"FELEM_CODE\":\"expression\"}]}";
StringBuffer response = new StringBuffer();

int return_code = g2config.addFeatureV2(config_handle, feature, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

### deleteFeature
Call G2Config's `deleteFeatureV2()` method. Returns 0 on success.

In [ ]:
String feature = "{\"FTYPE_CODE\":\"NAME\"}";

int return_code = g2config.deleteFeatureV2(config_handle, feature);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(return_code);

### addElementToFeature
Call G2Config's `addElementToFeatureV2()` method. Returns 0 on success

In [ ]:
String elmnt = "{\"FTYPE_CODE\":\"myNewFtype\", \"FELEM_CODE\":\"expression\",\"EXEC_ORDER\":4}";

int return_code = g2config.addElementToFeatureV2(config_handle, elmnt);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(return_code);

### deleteElementFromFeature
Call G2Config's `deleteElementFromFeatureV2()` method. Returns 0 on success

In [ ]:
String elmnt = "{\"FTYPE_CODE\":\"myNewFtype\", \"FELEM_CODE\":\"expression\"}";

int return_code = g2config.deleteElementFromFeatureV2(config_handle, elmnt);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(return_code);

### Standardization

#### listFeatureStandardizationFunctions
Call G2Config's `listFeatureStandardizationFunctionsV2()` method and print results.

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2config.listFeatureStandardizationFunctionsV2(config_handle, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

#### listFeaturesStandardizationFunctionCalls
Call G2Config's `listFeatureStandardizationFunctionCallsV2()` method and print results.

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2config.listFeatureStandardizationFunctionCallsV2(config_handle, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

#### getFeatureStandardizationFunctionCalls
Call G2Config's `getFeatureStandardizationFunctionCallV2()` method and print results.

In [ ]:
String featureStandard = "{\"SFCALL_ID\":4}";
StringBuffer response = new StringBuffer();

int return_code = g2config.getFeatureStandardizationFunctionCallV2(config_handle, featureStandard, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

#### addFeatureStandardizationFunctionCalls
Call G2Config's `addFeatureStandardizationFunctionCallV2()` method and print results.

In [ ]:
String featureStandard = "{\"FTYPE_CODE\":\"veh_vin\",\"SFCALL_ID\":789,\"SFUNC_CODE\":\"PARSE_ID\",\"EXEC_ORDER\":15}";
StringBuffer response = new StringBuffer();

int return_code = g2config.addFeatureStandardizationFunctionCallV2(config_handle, featureStandard, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

#### deleteFeatureStandardizationFunctionCall
Call G2Config's `deleteFeatureStandardizationFunctionCallV2()` method. Returns 0 on success

In [ ]:
String featureStandard = "{\"SFCALL_ID\":789}";

int return_code = g2config.deleteFeatureStandardizationFunctionCallV2(config_handle, featureStandard);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(return_code);

### Expression

#### listFeatureExpressionFunctions
Call G2Config's `listFeatureExpressionFunctionsV2()` method and print results.

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2config.listFeatureExpressionFunctionsV2(config_handle, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

#### listFeatureExpressionFunctionCalls
Call G2Config's `listFeatureExpressionFunctionCallsV2()` method and print results.

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2config.listFeatureExpressionFunctionCallsV2(config_handle, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

#### getFeatureExpressionFunctionCalls
Call G2Config's `getFeatureExpressionFunctionCallV2()` method and print results.

In [ ]:
String featureExpress = "{\"EFCALL_ID\":1}";
StringBuffer response = new StringBuffer();

int return_code = g2config.getFeatureExpressionFunctionCallV2(config_handle, featureExpress, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

#### addFeatureExpressionFunctionCall
Call G2Config's `addFeatureExpressionFunctionCallV2()` method and print results.

In [ ]:
String featureExpress = "{\"FTYPE_CODE\":\"veh_vin\",\"EFCALL_ID\":789,\"EFUNC_CODE\":\"EXPRESS_ID\",\"EXEC_ORDER\":15, \"BOM_ELEMENTS\":[{\"FTYPE_CODE\":\"veh_vin\",\"FELEM_CODE\":\"id_num\",\"REQUIRED\":\"Yes\"}]}";
StringBuffer response = new StringBuffer();

int return_code = g2config.addFeatureExpressionFunctionCallV2(config_handle, featureExpress, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

#### addFeatureExpressionFunctionCallElement
Call G2Config's `addFeatureExpressionFunctionCallElementV2()` method. Returns 0 on success.

In [ ]:
String featureExpressElm = "{\"EFCALL_ID\":789, \"FTYPE_CODE\":\"address\",\"FELEM_CODE\":\"addr5\",\"REQUIRED\":\"Yes\"}";

int return_code = g2config.addFeatureExpressionFunctionCallElementV2(config_handle, featureExpressElm);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(return_code);

#### deleteFeatureExpressionFunctionCallElement
Call G2Config's `deleteFeatureExpressionFunctionCallElementV2()` method. Returns 0 on success.

In [ ]:
String featureExpressElm = "{\"EFCALL_ID\":789, \"FTYPE_CODE\":\"ADDRESS\",\"FELEM_CODE\":\"ADDR5\"}";

int return_code = g2config.deleteFeatureExpressionFunctionCallElementV2(config_handle, featureExpressElm);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(return_code);

#### deleteFeatureExpressionFunctionCall
Call G2Config's `deleteFeatureExpressionFunctionCallV2()` method. Returns 0 on success

In [ ]:
String featureExpressElm = "{\"EFCALL_ID\":789}";

int return_code = g2config.deleteFeatureExpressionFunctionCallV2(config_handle, featureExpressElm);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(return_code);

### Comparison

#### listFeatureComparisonFunctions
Call G2Config's `listFeatureComparisonFunctionsV2()` method and print results.

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2config.listFeatureComparisonFunctionsV2(config_handle, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

#### listFeatureComparisonFunctionCalls
Call G2Config's `listFeatureComparisonFunctionCallsV2()` method and print results.

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2config.listFeatureComparisonFunctionCallsV2(config_handle, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

#### getFeatureComparisonFunctionCall
Call G2Config's `getFeatureComparisonFunctionCallV2()` method and print results.

In [ ]:
String featureCompFunc = "{\"CFCALL_ID\":2}";
StringBuffer response = new StringBuffer();

int return_code = g2config.getFeatureComparisonFunctionCallV2(config_handle, featureCompFunc, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

#### addFeatureComparisonFunctionCall
Call G2Config's `addFeatureComparisonFunctionCallV2()` method and print results.

In [ ]:
String featureCompFunc = "{\"FTYPE_CODE\":\"name_key\",\"CFCALL_ID\":789,\"CFUNC_CODE\":\"ID_COMP\",\"EXEC_ORDER\":15, \"BOM_ELEMENTS\":[{\"FELEM_CODE\":\"expression\"}]}";
StringBuffer response = new StringBuffer();

int return_code = g2config.addFeatureComparisonFunctionCallV2(config_handle, featureCompFunc, response);


if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

#### deleteFeatureComparisonFunctionCall
Call G2Config's `deleteFeatureComparisonFunctionCallV2()` method. Returns 0 on success.

In [ ]:
String featureComp = "{\"CFCALL_ID\":789}";

int return_code = g2config.deleteFeatureComparisonFunctionCallV2(config_handle, featureComp);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(return_code);

#### addFeatureComparisonFunctionCallElement
Call G2Config's `addFeatureComparisonFunctionCallElementV2()` method. Returns 0 on success

In [ ]:
String featureCompElm = "{\"CFCALL_ID\":5, \"FELEM_CODE\":\"addr6\"}";

int return_code = g2config.addFeatureComparisonFunctionCallElementV2(config_handle, featureCompElm);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(return_code);

#### deleteFeatureComparisonFunctionCallElement
Call G2Config's `deleteFeatureComparisonFunctionCallElementV2()` method. Returns 0 on success.

In [ ]:
String featureCompElm = "{\"CFCALL_ID\":5, \"FELEM_CODE\":\"ADDR4\"}";

int return_code = g2config.deleteFeatureComparisonFunctionCallElementV2(config_handle, featureCompElm);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(return_code);

### Distinct

#### listFeatureDistinctFunctions 
Call G2Config's `listFeatureDistinctFunctionV2()` method and print results.

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2config.listFeatureDistinctFunctionsV2(config_handle, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

#### listFeatureComparisonFunctionCalls
Call G2config's `listFeatureDistinctFunctionCallsV2()` method and print results

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2config.listFeatureDistinctFunctionsV2(config_handle, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

#### getFeatureDistinctFunctionCall
Call G2Config's `getFeatureDistinctFunctionCallV2()` method and print results.

In [ ]:
String featureFunc = "{\"DFCALL_ID\":2}";
StringBuffer response = new StringBuffer();

int return_code = g2config.getFeatureDistinctFunctionCallV2(config_handle, featureFunc, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

#### addFeatureDistinctFunctionCall
Call G2Config's `addFeatureDistinctFunctionCallV2()` method and print results.

In [ ]:
String featureFunc = "{\"FTYPE_CODE\":\"name_key\",\"DFCALL_ID\":789,\"DFUNC_CODE\":\"FELEM_STRICT_SUBSET\",\"EXEC_ORDER\":15, \"BOM_ELEMENTS\":[{\"FELEM_CODE\":\"expression\"}]}";
StringBuffer response = new StringBuffer();

int return_code = g2config.addFeatureDistinctFunctionCallV2(config_handle, featureFunc, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

#### deleteFeatureDistinctFunctionCall
Call G2Config's `deleteFeatureDistinctFunctionCallV2()` method. Returns 0 on success

In [ ]:
String featureFunc = "{\"DFCALL_ID\":1002}";

int return_code = g2config.deleteFeatureDistinctFunctionCallV2(config_handle, featureFunc);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(return_code);

#### addFeatureDistinctFunctionCallElement
Call G2Config's `addFeatureDistinctFunctionCallElementV2()` method. Returns 0 on success

In [ ]:
String featureElmt = "{\"DFCALL_ID\":5, \"FELEM_CODE\":\"addr6\"}";

int return_code = g2config.addFeatureDistinctFunctionCallElementV2(config_handle, featureElmt);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(return_code);

#### deleteFeatureDistinctFunctionCallElement
Call G2Config's `deleteFeatureDistrinctFunctionCallElementV2()` method. Returns 0 on success

In [ ]:
String featureElmt = "{\"DFCALL_ID\":5, \"FELEM_CODE\":\"ADDR4\"}";

int return_code = g2config.deleteFeatureDistinctFunctionCallElementV2(config_handle, featureElmt);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(return_code);

## Attributes

### listAttributeClasses
Call G2Config's `listAttributeClassesV2()` method and print results.

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2config.listAttributeClassesV2(config_handle, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

### listAttributes
Call G2Config's `listAttributesV2()` method and print results

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2config.listAttributesV2(config_handle, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

### getAttribute
Call G2Config's `getAttributeV2()` method and print results.

In [ ]:
String attr = "{\"ATTR_CODE\":\"DATA_SOURCE\"}";
StringBuffer response = new StringBuffer();

int return_code = g2config.getAttributeV2(config_handle, attr, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

### addAttribute
Call G2Config's `addAttributeV2()` method and print results.

In [ ]:
String attr = "{\"ATTR_CODE\":\"myNewAttr\",\"ATTR_ID\":789,\"ATTR_CLASS\":\"attribute\",\"FTYPE_CODE\":\"name_key\",\"FELEM_CODE\":\"latitude\",\"FELEM_REQ\":\"myReqValue\",\"DEFAULT_VALUE\":\"myDefault\",\"ADVANCED\":\"Y\",\"INTERNAL\":1}";
StringBuffer response = new StringBuffer();

int return_code = g2config.addAttributeV2(config_handle, attr, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);

### deleteAttribute
Call G2Config's `deleteAttributeV2()` method. Returns 0 on success

In [ ]:
String attr = "{\"ATTR_CODE\":\"NAME_FIRST\"}";

int return_code = g2config.deleteAttributeV2(config_handle, attr);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(response);